In [1]:
# parser for pdf scans inside xml-files from German Bundestag / Opendata for periods 1 - 18
# into the django parliament app
# https://www.bundestag.de/services/opendata

from __future__ import print_function
import os, sys
import django
from django.conf import settings
import re
import requests
import dataset
import datetime
from xml.etree import ElementTree
from urllib.parse import urljoin
# Extract agenda numbers not part of normdatei
from normality import normalize
import platform
import zipfile

if platform.node() == "srv-mcc-apsis":
    sys.path.append('/home/leey/tmv/BasicBrowser/')
    data_dir = '/home/leey/Plenarprotokolle'
else:
    # local paths
    sys.path.append('/home/leey/Documents/Data/tmv/BasicBrowser/')
    data_dir = '/home/leey/Documents/Data/Plenarprotokolle'

# imports and settings for django and database
# --------------------------------------------

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
# alternatively
#settings.configure(DEBUG=True)
django.setup()

# import from appended path
import parliament.models as pm
from parliament.tasks import do_search
import cities.models as cmodels

from parsing_utils import dehyphenate, POI, clean_text, correct_pdf_parsing_errors, search_party_names, search_person_party
from find_person_in_db_pdf import find_person_in_db
from regular_expressions_global import *

import pprint
pretty_printer = pprint.PrettyPrinter(indent=4)

/home/leey/venv/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
Slow version of gensim.models.doc2vec is being used
Slow version of Fasttext is being used


### Scraper

In [ ]:
# coding: utf-8
# ============================================================
# write output to file and terminal

time_stamp = datetime.datetime.now().strftime("%y%m%d_%H%M%S")
output_file = "./parlsessions_pdf_parser_output_" + time_stamp + ".log"
print("log file: {}".format(output_file))

class Logger(object):
    def __init__(self):
        self.terminal = sys.stdout
        self.log = open(output_file, "a")

    def write(self, message):
        self.terminal.write(message)
        self.log.write(message)

    def flush(self):
        #this flush method is needed for python 3 compatibility.
        #this handles the flush command by doing nothing.
        #you might want to specify some extra behavior here.
        pass


In [2]:
class SpeechParser(object):

    def __init__(self, lines, verbosity=0):
        self.lines = lines
        self.line_number = 0
        self.was_chair = True
        self.date = None
        self.verbosity = verbosity
        self.in_session = False
        self.in_header = False
        self.in_poi = False
        self.poi_content = ""
        self.poi_linecounter = 0
        self.chair = False
        self.text = []
        self.pars = []
        self.speaker = None
        self.speaker_party = None
        self.speaker_ortszusatz = None
        self.warnings_counter = 0

    def get_date(self):
        for line in self.lines:
            date_match = DATE.search(line)
            if date_match:
                try:
                    d = int(date_match.group(2))
                    m = int(D_MONTHS[date_match.group(3)])
                    y = int(date_match.group(4))
                    date = datetime.date(y, m, d)
                    self.date = date
                    return

                except ValueError:
                    print("date from manuscript not readable: {}".format(DATE.match(line)))
                    print("group 1: {}".format(DATE.match(line).group(1)))
                    print("group 2: {}".format(DATE.match(line).group(2)))
                    print("group 3: {}".format(DATE.match(line).group(3)))
                    print("group 4: {}".format(DATE.match(line).group(4)))
                    raise ValueError

        print("Parser: Did not find date")
        return None

    def append_text_and_poi(self):
        par = {
            'text': dehyphenate(self.text),
            'pois': []
        }
        for poi_raw in re.split(' [-–—]-? ', self.poi_content):
            poi_obj = POI(poi_raw)
            par['pois'].append(poi_obj)
            if self.verbosity > 0:
                print("interjection: speakers: {}, party: {}, speaker_party: {}, speaker_ortszusatz: {}, type: {},"
                      "\ninterjection text: {}".format(poi_obj.speakers, poi_obj.parties, poi_obj.speaker_party, poi_obj.speaker_ortszusatz, poi_obj.type, poi_obj.poitext))

        self.pars.append(par)
        self.text = []
        self.poi_content = ""

    def emit(self):
        data = {
            'speaker': self.speaker,
            'speaker_party': self.speaker_party,
            'speaker_ortszusatz': self.speaker_ortszusatz,
            'type': 'chair' if self.chair else 'speech',
            'pars': self.pars
        }
        self.was_chair = self.chair
        self.text = []
        self.pars = []
        if self.verbosity > 1:
            print("utterance: {}".format(data))
        return data

    def __iter__(self):

        no_lines = len(self.lines)
        self.line_number = -1

        while self.line_number + 1 < no_lines:

            self.line_number += 1
            line = self.lines[self.line_number].strip()
            if verbosity > 1:
                print("- l{l:04d}: ".format(l=self.line_number) + line)

            # Check if in session, session beginning, session ending
            if not self.in_session and BEGIN_MARK.match(line):
                print("= matched begin mark at line {}: {}".format(self.line_number, line))
                self.in_session = True
                continue
            if not self.in_session and INCOMPLETE_BEGIN_MARK.match(line):
                print("! warning at line {}: Matched only incomplete begin mark: {}".format(self.line_number, line))
                self.warnings_counter += 1
                self.in_session = True

            elif not self.in_session:
                continue

            if DISRUPTION_MARK.match(line):
                continue

            # empty line
            if not len(line):
                continue

            # match repeated mentioning of speaker from header
            if self.speaker:
                speaker = self.speaker.replace('Dr. ', '')
                speaker = speaker.split(" (")[0]
                speaker = speaker.split(", ")[0]
                # print("looking for {}".format(speaker))
                try:
                    SPEAKER_HEADER = re.compile('.{0,30}%s' % speaker)
                    if SPEAKER_HEADER.match(line):
                        if verbosity > 0:
                            print("= matched speaker in header: ", line)
                        continue
                except:
                    pass

            header_match = HEADER_MARK.match(line)
            if header_match is not None:
                if verbosity > 0:
                    print("= matched header: ", line)

                self.in_header = True
                continue

            if self.in_header and self.speaker is not None:
                if line.startswith(self.speaker):
                    if verbosity > 0:
                        print("= matched current speaker in header: {}".format(line))
                    continue
                else:
                    self.in_header = False

            is_top = False
            # new point on the agenda (top - tagesordnungspunkt)
            if TOP_MARK.match(line):
                if verbosity > 0:
                    print("= matched top mark: {}".format(line))
                is_top = True

            has_stopword = False
            for sw in SPEAKER_STOPWORDS:
                if sw.lower() in line.lower():
                    if verbosity > 0:
                        print("= setting stopword flag in line {}: {}".format(self.line_number, sw))
                    has_stopword = True

            # match speaker
            for k in range(1,4):
                lines = "\n".join(self.lines[self.line_number:self.line_number+k])
                lines = dehyphenate(lines, nl=True)
                # print(repr(lines)) # print with escape characters
                speaker_match = (PRESIDENT.match(lines) or
                                 PARTY_MEMBER_PDF.match(lines) or
                                 STAATSSEKR.match(lines) or
                                 STAATSMINISTER.match(lines) or
                                 WEHRBEAUFTRAGTER.match(lines) or
                                 BUNDESKANZLER.match(lines) or
                                 BEAUFTRAGT.match(lines) or
                                 MINISTER.match(lines) or
                                 BERICHTERSTATTER.match(lines))

                if speaker_match is not None:
                    # if matching interjection speakers instead of speech, exit
                    ij_match = POI_SPEAKER.search(speaker_match.string)
                    if ij_match:
                        speaker_match = None
                        pass
                
                    else:
                        if verbosity > 0:
                            print("= matched speaker at line {}: {}".format(self.line_number, speaker_match))
                        self.in_poi = False
                        self.line_number += k - 1
                        break


            if speaker_match is not None \
                    and not is_top \
                    and not has_stopword:

                if self.speaker is None and self.text == [] and self.pars == []:
                    pass
                else:
                    if len(self.pars) < 1 and self.text:
                        par = {
                            'text': dehyphenate(self.text),
                            'pois': []
                        }
                        self.pars.append(par)

                    # emit everything if a new speaker has been identified
                    yield self.emit()

                role = line.strip().split(' ')[0]
                self.speaker = speaker_match.group(0).strip(' :')
                self.speaker_party = search_person_party(line.strip().split(':')[0])
                if speaker_match.group(2) is not None:
                    try:
                        self.speaker_ortszusatz = REMOVE_BRACKET.match(speaker_match.group(2)).group(1)
                    except AttributeError:
                        pass
                else:
                    self.speaker_ortszusatz = None
                self.chair = role in CHAIRS

                # if a new speaker has been identified, add the remainder of the text of that line to self.text
                self.text = [':'.join(lines.split(':')[1:])]

                continue


            # match end mark
            for k in range(1,3):
                lines = "\n".join(self.lines[self.line_number:self.line_number+k])
                lines = dehyphenate(lines)
                if END_MARK.search(lines):
                    print("= matched end mark at line {}: {}".format(self.line_number, lines))
                    self.text.append(lines)
                    par = {
                        'text': dehyphenate(self.text),
                        'pois': []
                    }
                    self.pars.append(par)
                    yield self.emit()
                    return

            # match interjections
            poi_match = POI_MARK.match(line)
            if poi_match is not None:
                self.poi_content = poi_match.group(1)
                self.append_text_and_poi()
                continue

            if not self.in_poi:
                poi_begin = POI_BEGIN.match(line)
                if poi_begin:
                    if verbosity > 1:
                        print("= raised in_poi flag")
                    self.in_poi = True
                    self.poi_content = line
                    self.poi_linecounter = 0
                    continue
            else:
                self.poi_content += "\n" + line
                self.poi_linecounter += 1
                if POI_END.match(line):
                    self.poi_content = dehyphenate(self.poi_content).strip().strip('()')
                    self.append_text_and_poi()
                    if verbosity > 1:
                        print("= matched poi end")
                    self.in_poi = False
                if self.poi_linecounter > 10:
                    print("! Warning: No match of poi end after 10 lines. Going back to normal mode.")
                    self.warnings_counter += 1
                    self.in_poi = False
                    self.text.append(self.poi_content)
                continue

            self.text.append(line)

        print("! Warning: Reached end of file without end mark")
        self.warnings_counter += 1
        yield self.emit()


def file_metadata(filename):
    fname = os.path.basename(filename)
    try:
        return int(fname[:2]), int(fname[2:5])
    except:
        return int(fname[:2]), fname[2:5]


def german_date(str):
    if str is None:
        return None
    return datetime.datetime.strptime(str,"%d.%m.%Y").date()

In [3]:
# ====================================================================
# ========== parse function ==========================================
# ====================================================================


def parse_transcript(file, verbosity=1):

    warnings_counter2 = 0
    if isinstance(file, str):
        print("loading text from {}".format(file))
        try:
            with open(file) as fh:
                content = fh.read()
                text = clean_text(content)
        except UnicodeDecodeError:
            print("Reloading in other encoding (windows-1252)")
            with open(file, encoding="windows-1252") as fh:
                content = fh.read()
                text = clean_text(content)
        filename = file
        wp, session = file_metadata(filename)

    # open file in zip archive
    elif isinstance(file, zipfile.ZipExtFile):
        content = file.read()
        filename = file.name
        if filename.endswith(".xml"):
            root = ElementTree.fromstring(content)
            if verbosity > 0:
                print("loading text from {}".format(filename))

            # display contents of xml file
            if verbosity > 1:
                print("xml root: {}, attributes: {}".format(root.tag, root.attrib))
                for child in root:
                    print("xml child: {}, attributes: {}".format(child.tag, child.attrib))
                    print("xml beginning of text: {}".format(child.text[:100].replace('\n', ' ')))

            wp = int(root.find("WAHLPERIODE").text)
            document_type = root.find("DOKUMENTART").text
            if document_type != "PLENARPROTOKOLL":
                print("Warning: document {} is not tagged as Plenarprotokoll but {}".format(filename, document_type))
                warnings_counter2 += 1
            number = root.find("NR").text
            session = int(number.split("/")[1])
            date = root.find("DATUM").text
            titel = root.find("TITEL").text
            text = clean_text(root.find("TEXT").text)
            text = correct_pdf_parsing_errors(text)
        else:
            print("filetype not xml")
            return 0

    else:
        print("invalid filetype")
        return 0

    base_data = {
        'filename': filename,
        'sitzung': session,
        'wahlperiode': wp
    }

    print("\nParsing transcript: {}/{}, from {}".format(wp, session, filename))
    utterance_counter = 0
    paragraph_counter = 0
    interjection_counter = 0

    # start parsing
    parser = SpeechParser(text.split('\n'), verbosity=verbosity)
    # get_date is not working for all documents
    parser.get_date()
    if isinstance(file, zipfile.ZipExtFile):
        if parser.date != german_date(date):
            print("! Warning: dates do not match")
            warnings_counter2 += 1
            print(parser.date)
            print(date)
            print(german_date(date))

    parl, created = pm.Parl.objects.get_or_create(
        country=cmodels.Country.objects.get(name="Germany"),
        level='N'
    )
    if created and verbosity > 0:
        print("created new object for parliament")

    pp, created = pm.ParlPeriod.objects.get_or_create(
                                    parliament=parl,
                                    n=wp)
    if created and verbosity > 0:
        print("created new object for legislative period")

    doc, created = pm.Document.objects.get_or_create(
        parlperiod=pp,
        doc_type="Plenarprotokoll",
        date=german_date(date),
        sitting=session,
        text_source="from https://www.bundestag.de/service/opendata (scans of pdfs with xml metadata)"
    )
    if created:
        print("created new object for plenary session document")
    doc.save()

    doc.utterance_set.all().delete()

    # parser.__iter__ yields dict with paragraphs + speaker + speaker_party + interjections (poi)
    for contrib in parser:

        if verbosity > 1:
            print("saving utterance: {}".format(contrib))

        # update dictionary
        contrib.update(base_data)

        if contrib['speaker']:
            info_dict = {'wp': wp, 'session': session, 'speaker_party': contrib['speaker_party'],
                         'speaker_ortszusatz': contrib['speaker_ortszusatz'], 'source_type': 'PDF/SP'}
            per = find_person_in_db(contrib['speaker'], add_info=info_dict, verbosity=verbosity)
        else:
            print("! Warning: No speaker given, not saving the following contribution: {}".format(contrib))
            warnings_counter2 += 1
            continue

        if per is None:
            print("! Warning: Not able to match person, not saving the following contribution: {}".format(contrib))
            warnings_counter2 += 1
            continue

        position = PERSON_POSITION.search(contrib['speaker'])

        if position:
            role_description = position.group(0)

            speaker_role_set = pm.SpeakerRole.objects.filter(alt_names__contains=[role_description])
            if len(speaker_role_set) < 1:
                speaker_role = pm.SpeakerRole(name=role_description, alt_names=[role_description])
                speaker_role.save()
            else:
                speaker_role = speaker_role_set.first()
                if len(speaker_role_set) > 1:
                    print("Warning: several speaker roles matching")

            ut = pm.Utterance(
                document=doc,
                speaker=per,
                speaker_role=speaker_role
            )
        else:
            ut = pm.Utterance(
                document=doc,
                speaker=per
            )

        ut.save()
        utterance_counter += 1

        for par in contrib['pars']:

            if par['text']:
                para = pm.Paragraph(
                    utterance=ut,
                    text=par['text'],
                    word_count=len(par['text'].split()),
                    char_len=len(par['text'])
                )
                para.save()
                paragraph_counter += 1
            else:
                print("! Warning: Empty paragraph ({})".format(par))
                warnings_counter2 += 1
                for ij in par['pois']:
                    print("poi: {}".format(ij.poitext))
                continue

            for ij in par['pois']:
                if ij.type is None:
                    print("! Warning: Ommiting interjection. Interjection type not identified for: {}".format(ij.poitext))
                    warnings_counter2 += 1
                    continue
                interjection = pm.Interjection(
                    paragraph=para,
                    text=ij.poitext,
                    type=ij.type
                )
                interjection.save()
                interjection_counter += 1

                if ij.parties:
                    for party_name in ij.parties.split(':'):
                        party, created = pm.Party.objects.get_or_create(
                            name=party_name
                        )

                        interjection.parties.add(party)
                if ij.speakers:
                    for person in ij.speakers:
                        info_dict = {'wp': wp, 'session': session, 'speaker_party': contrib['speaker_party'],
                                     'speaker_ortszusatz': contrib['speaker_ortszusatz'], 'source_type': 'PDF/SP'}
                        per = find_person_in_db(person, add_info=info_dict, verbosity=verbosity)
                        if per is not None:
                            interjection.persons.add(per)
                        else:
                            print("! Warning: Speaker could not be identified")
                            warnings_counter2 += 1

    if not parser.in_session:
        print("! Error: beginning of session not found")
        return (1, 0)

    print("==================================================")
    print("Summary for {}:".format(filename))
    print("number of utterances: {}".format(utterance_counter))
    print("number of paragraphs: {}".format(paragraph_counter))
    print("number of interjections: {}".format(interjection_counter))
    print("warnings in SpeechParser generator: {}".format(parser.warnings_counter))
    print("warnings in parse_transcript function: {}".format(warnings_counter2))
    print("==================================================")

    if utterance_counter <= 0:
        return (1, 0)
    else:
        return (0, parser.warnings_counter + warnings_counter2)

# ==========================================================================================================
# ==========================================================================================================
# what is this for? 
#def clear_db():
#    database = dataset.connect(db)
#    table = database['plpr']
#    table.delete()

# =================================================================================================================
# =================================================================================================================


def lines_with_one_character(file):

    if isinstance(file, str):
        # print("loading text from {}".format(file))
        with open(file) as fh:
            text = fh.read()
        text = text.replace("\t", "").split("\n")

    # open file in zip archive
    elif isinstance(file, zipfile.ZipExtFile):
        content = file.read()
        filename = file.name
        if filename.endswith(".xml"):
            root = ElementTree.fromstring(content)
            text = root.find("TEXT").text.replace("\t", "").split("\n")
            # print("loading text from {}".format(filename))
        else:
            print("filetype not xml")
            return None
        file.close()

    text = [line.strip() for line in text if line.strip() != '']
    count = sum([1 for line in text if len(line) == 1])

    return count

In [ ]:
# =================================================================================================================

# main execution script
if __name__ == '__main__':

    sys.stdout = Logger()

    # settings for parsing
    delete_additional_persons = False
    delete_all = False
    verbosity = 0

    if delete_all:
        print("Deleting all documents, utterances, paragraphs and interjections.")
        # pmodels.Person.objects.all().delete()
        # pmodels.Parl.objects.all().delete()
        # pmodels.ParlPeriod.objects.all().delete()
        pm.Interjection.objects.all().delete()
        pm.Paragraph.objects.all().delete()
        pm.Utterance.objects.all().delete()
        pm.Document.objects.all().delete()
        print("Deletion done.")

    if delete_additional_persons:
        print("Deleting all persons added from protocol parsing.")
        pm.Person.objects.filter(information_source__startswith="from protocol scraping").delete()

    document_counter = 0
    count_errors = 0
    count_warnings_docs = 0
    count_warnings_sum = 0

    wps = range(12, 11, -1)
    sessions = range(13, 245)

    print("start parsing...")
    for wp in wps:
        collection = "pp{wp:02d}-data.zip".format(wp=wp)
        print(collection)

        archive = zipfile.ZipFile(os.path.join(data_dir, collection), 'r')
        print("loading files from {}".format(collection))
        filelist = [fzip.filename for fzip in archive.infolist()]

        for session in sessions:
            filename = "{wp:02d}{s:03d}.xml".format(wp=wp, s=session)
            if filename in filelist:

                # delete old protocol
                pm.Document.objects.filter(parlperiod__n=wp, sitting=session,
                                           text_source="from https://www.bundestag.de/service/opendata "
                                                       "(scans of pdfs with xml metadata)").delete()
                f = archive.open(filename)
                print(f)
                parser_errors, parser_warnings = parse_transcript(f, verbosity=verbosity)
                count_errors += parser_errors
                if parser_warnings > 0:
                    count_warnings_docs += 1
                    count_warnings_sum += parser_warnings

                document_counter += 1
                f.close()

                f = archive.open(filename)
                print("lines with one character: {}".format(lines_with_one_character(f)))
                print("==================================================\n")

                f.close()
            else:
                print("{} not in archive".format(filename))

        archive.close()


    print("\n==================================================")
    print("Summary for {} documents:".format(document_counter))
    print("Documents with errors: {}".format(count_errors))
    print("Documents with warnings: {}".format(count_warnings_docs))
    print("Sum of all warnings: {}".format(count_warnings_sum))

### Testing

In [3]:
wps = range(11, 10, -1)
sessions = range(4, 5)

wp = 11
session = 4

testdata_dir = "../data/"
filename = "{wp:02d}{s:03d}.xml".format(wp=wp, s=session)
test_filename = "{wp:02d}{s:03d}_testedit.xml".format(wp=wp, s=session)
file = testdata_dir + filename
print(file)

../data/11004.xml


In [4]:
verbosity = 1
print("loading text from {}".format(file))
try:
    with open(file) as fh:
        content = fh.read()
        text = clean_text(content)
except UnicodeDecodeError:
    print("Reloading in other encoding (windows-1252)")
    with open(file, encoding="windows-1252") as fh:
        content = fh.read()
        text = clean_text(content)
filename = file
wp, session = file_metadata(filename)

base_data = {
    'filename': filename,
    'sitzung': session,
    'wahlperiode': wp
}

print("\nParsing transcript: {}/{}, from {}".format(wp, session, filename))
utterance_counter = 0
paragraph_counter = 0
interjection_counter = 0

parser = SpeechParser(text.split('\n'), verbosity=verbosity)

loading text from ../data/11004.xml

Parsing transcript: 11/4, from ../data/11004.xml


In [5]:
# utterances
for contrib in parser:
    if contrib['speaker']:
        print(contrib['speaker'])
        print(contrib['speaker_party'])
        print(contrib['speaker_ortszusatz'])
        print(contrib['pars'])
        print('####')

= matched begin mark at line 45: Beginn: 10.00 Uhr
= matched speaker at line 46: <re.Match object; span=(0, 25), match='Präsident Dr. Jenninger: '>
interjection: speakers: [], party: , speaker_party: , speaker_ortszusatz: , type: 7,
interjection text: Beif all
= matched top mark: Meine Damen und Herren, die heutige Tagesordnung wird gemäß § 39 unserer Geschäftsordnung um den Zusatzpunkt
= matched top mark: Ich rufe den Tagesordnungspunkt 1 auf:
= setting stopword flag in line 54: ich rufe den
= matched speaker at line 58: <re.Match object; span=(0, 25), match='Dr. Kohl, Bundeskanzler: '>
Präsident Dr. Jenninger
None
None
[{'text': ' Die Sitzung ist eröffnet. Meine Damen und Herren, heute feiert der Abgeordnete Bahr seinen 65. Geburtstag. Ich darf ihm die besten Wünsche des Hauses übermitteln.', 'pois': [<parsing_utils.POI object at 0x7ff1e71a90b8>]}]
####
= matched speaker at line 60: <re.Match object; span=(0, 27), match='Kleinert [Marburg] [GRÜNE]:'>
Dr. Kohl, Bundeskanzler
None
None

interjection text: Beifall bei der CDU/CSU und der FDP
interjection: speakers: [], party: None, speaker_party: , speaker_ortszusatz: , type: 1,
interjection text: Beifall bei Abgeordneten der GRÜNEN
interjection: speakers: ['Dr. Daniels'], party: , speaker_party: None, speaker_ortszusatz: Regensburg, type: 2,
interjection text: Aber nicht bei Ihrem Programm!
= matched header:  Deutscher Bundestag – 11. Wahlperiode – 4. Sitzung. Bonn, Mittwoch, den 18. März 1987	73
interjection: speakers: [], party: cducsu:fdp, speaker_party: , speaker_ortszusatz: , type: 1,
interjection text: Beifall bei der CDU/CSU und der FDP
interjection: speakers: [], party: cducsu:fdp, speaker_party: , speaker_ortszusatz: , type: 1,
interjection text: Beifall bei der CDU/CSU und der FDP
interjection: speakers: [], party: None, speaker_party: , speaker_ortszusatz: , type: 5,
interjection text: Lachen bei den GRÜNEN
interjection: speakers: [], party: , speaker_party: , speaker_ortszusatz: , type: 2,
interjection tex

interjection text: Beifall bei der CDU/CSU und der FDP
interjection: speakers: ['Duve'], party: , speaker_party: spd, speaker_ortszusatz: , type: 2,
interjection text: Eingegangen ist er!
= matched header:  Deutscher Bundestag – 11. Wahlperiode – 4. Sitzung. Bonn, Mittwoch, den 18. März .1987	91
interjection: speakers: [], party: , speaker_party: , speaker_ortszusatz: , type: 2,
interjection text: Ja, 2,4 Millionen Arbeitslose!
interjection: speakers: [], party: cducsu, speaker_party: , speaker_ortszusatz: , type: 1,
interjection text: Beifall bei der CDU/CSU
interjection: speakers: [], party: , speaker_party: , speaker_ortszusatz: , type: 7,
interjection text: Sehr wahr! bei der CDU/CSU
interjection: speakers: ['Dr. Dregger'], party: , speaker_party: cducsu, speaker_ortszusatz: , type: 2,
interjection text: So ist es!
interjection: speakers: ['Dr. Dregger'], party: , speaker_party: cducsu, speaker_ortszusatz: , type: 2,
interjection text: So ist es!
interjection: speakers: ['Dr. Apel'

interjection text: Beifall bei der FDP und der CDU/CSU Zurufe von der SPD und den GRÜNEN
interjection: speakers: ['Wieczorek'], party: , speaker_party: spd, speaker_ortszusatz: Duisburg, type: 2,
interjection text: Aber mit den Arbeitslosen nicht!
interjection: speakers: [], party: , speaker_party: , speaker_ortszusatz: , type: 2,
interjection text: Reden Sie mal über Stahl!
interjection: speakers: [], party: spd, speaker_party: , speaker_ortszusatz: , type: 6,
interjection text: Weitere Zurufe von der SPD
interjection: speakers: [], party: , speaker_party: , speaker_ortszusatz: , type: 7,
interjection text: Sehr gut! bei der CDU/CSU
interjection: speakers: [], party: spd, speaker_party: , speaker_ortszusatz: , type: 6,
interjection text: Zurufe von der SPD
interjection: speakers: ['Lambinus'], party: , speaker_party: spd, speaker_ortszusatz: , type: 2,
interjection text: Mit euch! Wer war damals Wirtschaftsminister?
interjection: speakers: [], party: spd, speaker_party: , speaker_orts

interjection: speakers: ['Kittelmann'], party: , speaker_party: cducsu, speaker_ortszusatz: , type: 2,
interjection text: Abwarten!
= setting stopword flag in line 2613: zitiert
= matched header:  Deutscher Bundestag – 11. Wahlperiode – 4. Sitzung. Bonn, Mittwoch, den 18. März 1987	127
= setting stopword flag in line 2620: fordert
= setting stopword flag in line 2626: fordert
= matched speaker in header:  Vizepräsident Frau Renger: Herr Kollege, ich glaube, Sie sollten zum Ende kommen.
interjection: speakers: ['Kittelmann'], party: , speaker_party: cducsu, speaker_ortszusatz: , type: 2,
interjection text: Sie haben es nötig!
= setting stopword flag in line 2632: fordert
interjection: speakers: [], party: spd, speaker_party: , speaker_ortszusatz: , type: 1,
interjection text: Beifall bei der SPD
= matched speaker in header:  Vizepräsident Frau Renger: Das Wort hat Herr Abgeordneter Graf Lambsdorff.
= setting stopword flag in line 2636: fordert
= matched header:  128	Deutscher Bundestag 

In [6]:
# interjections
for contrib in parser:
    for par in contrib['pars']:
        print(par['pois'])
        print("####")

= matched speaker in header:  Vizepräsident Frau Renger  	120 C
= matched header:  Deutscher Bundestag – 11. Wahlperiode – 4. Sitzung. Bonn, Mittwoch, den 18. März .1987	51
= matched speaker at line 46: <re.Match object; span=(0, 25), match='Präsident Dr. Jenninger: '>
[]
####
interjection: speakers: [], party: , speaker_party: , speaker_ortszusatz: , type: 7,
interjection text: Beif all
= matched top mark: Meine Damen und Herren, die heutige Tagesordnung wird gemäß § 39 unserer Geschäftsordnung um den Zusatzpunkt
= matched top mark: Ich rufe den Tagesordnungspunkt 1 auf:
= setting stopword flag in line 54: ich rufe den
= matched speaker at line 58: <re.Match object; span=(0, 25), match='Dr. Kohl, Bundeskanzler: '>
####
= matched speaker at line 60: <re.Match object; span=(0, 27), match='Kleinert [Marburg] [GRÜNE]:'>
[]
####
interjection: speakers: [], party: cducsu:fdp, speaker_party: , speaker_ortszusatz: , type: 1,
interjection text: Beifall bei der CDU/CSU und der FDP
interjection:

interjection text: Beifall bei der CDU/CSU, der FDP sowie bei Abgeordneten der SPD
interjection: speakers: [], party: cducsu:fdp:spd, speaker_party: , speaker_ortszusatz: , type: 1,
interjection text: Erneuter Beifall bei der CDU/CSU, der FDP sowie bei Abgeordneten der SPD
interjection: speakers: ['Jahn'], party: , speaker_party: spd, speaker_ortszusatz: Marburg, type: 2,
interjection text: Wessen Erwartungen?
= matched header:  68	Deutscher Bundestag – 11. Wahlperiode – 4. Sitzung. Bonn, Mittwoch, den 18. März 1987
interjection: speakers: [], party: cducsu:fdp, speaker_party: , speaker_ortszusatz: , type: 1,
interjection text: Beifall bei der CDU/CSU und der FDP
interjection: speakers: [], party: , speaker_party: , speaker_ortszusatz: , type: 7,
interjection text: Zustimmung bei der CDU/CSU
interjection: speakers: [], party: cducsu:fdp, speaker_party: , speaker_ortszusatz: , type: 1,
interjection text: Beifall bei der CDU/CSU und der FDP
= matched header:  Deutscher Bundestag – 11. Wa

interjection text: Na, na!
interjection: speakers: ['Frau Fuchs'], party: , speaker_party: spd, speaker_ortszusatz: Köln, type: 2,
interjection text: Auf Pump haben Sie sie bezahlt, nicht wir!
interjection: speakers: ['Frau Fuchs'], party: , speaker_party: spd, speaker_ortszusatz: Köln, type: 2,
interjection text: Ich freue mich darauf!
interjection: speakers: [], party: cducsu, speaker_party: , speaker_ortszusatz: , type: 1,
interjection text: Beifall bei der CDU/CSU
interjection: speakers: ['Dr. Vogel'], party: , speaker_party: spd, speaker_ortszusatz: , type: 2,
interjection text: Wie feinfühlig, Herr Waigel!
interjection: speakers: ['Dr. Bötsch'], party: , speaker_party: cducsu, speaker_ortszusatz: , type: 2,
interjection text: Sehr richtig!
interjection: speakers: [], party: cducsu, speaker_party: , speaker_ortszusatz: , type: 1,
interjection text: Beifall bei der CDU/CSU
interjection: speakers: [], party: , speaker_party: , speaker_ortszusatz: , type: 2,
interjection text: Aber n

interjection text: Beifall bei der FDP und der CDU/CSU
interjection: speakers: [], party: None, speaker_party: , speaker_ortszusatz: , type: 6,
interjection text: Zuruf von den GRÜNEN:. Wie?
interjection: speakers: [], party: , speaker_party: , speaker_ortszusatz: , type: 2,
interjection text: Sprücheklopfer!
interjection: speakers: ['Dr. Soell'], party: , speaker_party: spd, speaker_ortszusatz: , type: 2,
interjection text: Und die Subventionsliste in der Kiste?
= matched speaker at line 1898: <re.Match object; span=(0, 26), match='Vizepräsident Cronenberg: '>
####
####
####
####
[<parsing_utils.POI object at 0x7ff1e71cb5c0>, <parsing_utils.POI object at 0x7ff1e71cbac8>, <parsing_utils.POI object at 0x7ff1e71cb630>]
####
####
= matched header:  Deutscher Bundestag – 11. Wahlperiode – 4. Sitzung. Bonn, Mittwoch, den 18. März 1987	105
= matched speaker at line 1900: <re.Match object; span=(0, 46), match='Dr. Bangemann, Bundesminister für Wirtschaft: '>
[]
####
interjection: speakers: ['

interjection text: Beifall bei den GRÜNEN
= setting stopword flag in line 2407: zitiert
interjection: speakers: [], party: None, speaker_party: , speaker_ortszusatz: , type: 1,
interjection text: Heiterkeit und Beifall bei den GRÜNEN
= setting stopword flag in line 2409: fordern
interjection: speakers: ['Dr. Bötsch'], party: None, speaker_party: , speaker_ortszusatz: , type: 6,
interjection text: Zuruf des Abg. Dr. Bötsch [CDU/CSU]
= matched header:  120	Deutscher Bundestag – 11. Wahlperiode – 4. Sitzung. Bonn, Mittwoch, den 18. März 1987
interjection: speakers: [], party: None, speaker_party: , speaker_ortszusatz: , type: 1,
interjection text: Beifall bei den GRÜNEN
interjection: speakers: ['Kittelmann'], party: , speaker_party: cducsu, speaker_ortszusatz: , type: 2,
interjection text: Sie spinnen!
interjection: speakers: ['Dr. Bötsch'], party: , speaker_party: cducsu, speaker_ortszusatz: , type: 2,
interjection text: Ich bin aus Süddeutschland!
interjection: speakers: [], party: None

In [7]:
# interjections info
for contrib in parser:
    for par in contrib['pars']:
        for ij in par['pois']:
            if ij.speakers:
                for person in ij.speakers:
                    #print(person)
                    print(ij.speaker_party)
                    print(ij.speaker_ortszusatz)
                    print('###')

= matched speaker in header:  Vizepräsident Frau Renger  	120 C
= matched header:  Deutscher Bundestag – 11. Wahlperiode – 4. Sitzung. Bonn, Mittwoch, den 18. März .1987	51
= matched speaker at line 46: <re.Match object; span=(0, 25), match='Präsident Dr. Jenninger: '>
interjection: speakers: [], party: , speaker_party: , speaker_ortszusatz: , type: 7,
interjection text: Beif all
= matched top mark: Meine Damen und Herren, die heutige Tagesordnung wird gemäß § 39 unserer Geschäftsordnung um den Zusatzpunkt
= matched top mark: Ich rufe den Tagesordnungspunkt 1 auf:
= setting stopword flag in line 54: ich rufe den
= matched speaker at line 58: <re.Match object; span=(0, 25), match='Dr. Kohl, Bundeskanzler: '>
= matched speaker at line 60: <re.Match object; span=(0, 27), match='Kleinert [Marburg] [GRÜNE]:'>
interjection: speakers: [], party: cducsu:fdp, speaker_party: , speaker_ortszusatz: , type: 1,
interjection text: Beifall bei der CDU/CSU und der FDP
interjection: speakers: ['Kleinert

interjection text: Unterbrechung von 12.34 bis 14.00 Uhr
= matched speaker at line 822: <re.Match object; span=(0, 24), match='Vizepräsident Stücklen: '>
= matched top mark: Ich rufe den Tagesordnungspunkt 2 auf:
= setting stopword flag in line 823: ich rufe den
= matched top mark: Meine Damen und Herren, zu diesem Tagesordnungspunkt liegen ein Antrag der Fraktionen der CDU/CSU, SPD und FDP auf Drucksache 11/53 sowie ein Antrag der Fraktion DIE GRÜNEN auf Drucksache 11/55 vor. Eine Aussprache ist nicht vorgesehen.
interjection: speakers: ['Kleinert'], party: , speaker_party: , speaker_ortszusatz: , type: 7,
interjection text: Abg. Kleinert [Marburg] [GRÜNE] meldet sich zu Wort
= setting stopword flag in line 830: Ich möchte
interjection: speakers: [], party: None, speaker_party: , speaker_ortszusatz: , type: 1,
interjection text: Beifall bei den GRÜNEN
= matched speaker in header:  Vizepräsident Stücklen: Zur Geschäftsordnung, Herr Abgeordneter Dr. Bötsch.
= matched header:  74	Deutsch

interjection text: Gott sei Dank!
interjection: speakers: [], party: cducsu, speaker_party: , speaker_ortszusatz: , type: 1,
interjection text: Beifall bei Abgeordneten der CDU/CSU
interjection: speakers: ['Roth'], party: , speaker_party: spd, speaker_ortszusatz: , type: 2,
interjection text: Sie verteilen doch Steuerreduktion vorher!
interjection: speakers: ['Roth'], party: , speaker_party: spd, speaker_ortszusatz: , type: 2,
interjection text: Das war jetzt zehntklassig!
= matched speaker at line 1478: <re.Match object; span=(0, 53), match='Zuruf von der SPD: Immer auf den Späth! –\nRoth [>
spd

###
spd

###
spd

###
interjection: speakers: [], party: , speaker_party: , speaker_ortszusatz: , type: 7,
interjection text: Hört! Hört! bei den GRÜNEN
interjection: speakers: [], party: cducsu:fdp, speaker_party: , speaker_ortszusatz: , type: 1,
interjection text: Beifall bei der CDU/CSU und der FDP
= matched speaker at line 1491: <re.Match object; span=(0, 11), match='Roth [SPD]:'>
interje

= matched speaker at line 2037: <re.Match object; span=(0, 42), match='Unruhe bei der SPD – Jahn [Marburg]\n[SPD]:'>
spd

###
spd

###
spd

###
spd

###
None
Marburg
###
interjection: speakers: ['Vosen'], party: , speaker_party: , speaker_ortszusatz: , type: 7,
interjection text: Abg. Vosen [SPD] meldet sich zu einer Zwischenfrage
interjection: speakers: ['Dr. Vogel'], party: , speaker_party: spd, speaker_ortszusatz: , type: 2,
interjection text: Wer hat denn davon gesprochen?
interjection: speakers: ['Dr. Vogel'], party: , speaker_party: spd, speaker_ortszusatz: , type: 2,
interjection text: Hören Sie einmal zu! Putzen Sie die Ohren!
interjection: speakers: ['Jahn'], party: , speaker_party: spd, speaker_ortszusatz: Marburg, type: 2,
interjection text: Mit Recht kritisiert, jawohl!
= matched speaker at line 2053: <re.Match object; span=(0, 27), match='Kleinert [Marburg] [GRÜNE]:'>


###
spd

###
spd

###
spd
Marburg
###
interjection: speakers: [], party: cducsu:fdp, speaker_party: , sp

interjection: speakers: ['Frau Unruh'], party: , speaker_party: None, speaker_ortszusatz: , type: 2,
interjection text: Sie träumen am meisten!
interjection: speakers: ['Roth'], party: , speaker_party: spd, speaker_ortszusatz: , type: 2,
interjection text: Genau so wurde das EWS angegriffen!
= setting stopword flag in line 2673: fordert
interjection: speakers: ['Zuruf von den'], party: , speaker_party: None, speaker_ortszusatz: , type: 2,
interjection text: Aha!
interjection: speakers: [], party: , speaker_party: , speaker_ortszusatz: , type: 7,
interjection text: Zustimmung bei der FDP und der SPD
interjection: speakers: ['Roth'], party: , speaker_party: spd, speaker_ortszusatz: , type: 2,
interjection text: Natürlich!
= matched header:  130	Deutscher Bundestag – 11. Wahlperiode – 4. Sitzung. Bonn, Mittwoch, den 18. März 1987
= matched speaker at line 2685: <re.Match object; span=(0, 25), match='der CDU/CSU – Roth [SPD]:'>
None

###
cducsu

###
spd

###
spd

###
spd

###
spd

###
cduc

---

#### Identifying parties

In [8]:
test_string = "Dr. Vogel [SPD]:"
test_string2 = 'Wieczorek [Duisburg] [SPD]:'
#test_string = normalize(test_string)
test_party = test_string.strip().split(' ')[-1]

In [10]:
test_party2 = 'F.D.P.'
test_person = 'Rühe (CDU/CSU)'

In [11]:
PARTIES_REGEX_PDF = {
    'cducsu': re.compile(' ?cdu ?(csu)?'),
    'spd': re.compile(' ?spd'),
    'linke': re.compile(' ?(die|der|den) linken?| pds'),
    'fdp': re.compile(' ?fdp|F.D.P.'),
    'gruene': re.compile(' ?bund ?nis\-?(ses)? ?90 die gru ?nen'),
    'afd': re.compile(' ?AfD')
}

In [13]:
def search_person_party(text):
    """
    returns the party of a speaker from an input string (from raw text)
    """
    if text is None:
        return
    text = normalize(text)
    # identify correct group of text in name
    
    # find party
    for party, rex in PARTIES_REGEX.items():
        if rex.findall(text):
            person_party = party
            return person_party

In [14]:
search_person_party(test_person)

'cducsu'

#### Interjections

In [6]:
# from class POI in parsing_utils.py 
text2 = 'Wieczorek [Duisburg] [SPD]: '
if ": " in text2:
    sinfo = text2.split(': ', 1)
    speaker = sinfo[0].split(' ')
    #if len(speaker) > 1:
    #    self.speakers.append(speaker[0].strip())
print(speaker)
search_person_party(sinfo[0])

['Wieczorek', '[Duisburg]', '[SPD]']


'spd'

In [46]:
REMOVE_BRACKET = re.compile('[\(\[](.*)[\)\]]')
test_orts = '[München]'
REMOVE_BRACKET.match(test_orts).group(1)

'München'

In [10]:
speaker_match = PARTY_MEMBER_PDF.match(sinfo[0])
REMOVE_BRACKET.match(speaker_match.group(2)).group(1)

str

#### Regular Expressions

In [19]:
# groups name, ortszusatz (if any), and party of a speaker
PARTY_MEMBER_PDF = re.compile('([^\(\)]{2,50}?)([\[\(][^\(\)]*[\]\)])?\s?([\[\(][^\(\)]*[\]\)])\s?:')
PARTY_MEMBER_PDF_POI = re.compile('[\(\)]?\s?(\w{2,50}?)\s([\[\(][^\(\)]*[\]\)])?\s?([\[\(][^\(\)]*[\]\)])\s?')

In [25]:
test_result = PARTY_MEMBER_PDF.match(test_string2).group(3)
speaker_party = normalize(test_result)
speaker_party = ' '
test_result

'[Duisburg] [SPD]'

In [22]:
name = clean_text(test_string)
name = INHYPHEN.sub(r'\1\2', name)
name = name.replace('\n', ' ')
name = name.rstrip(':') #new
name = NAME_REMOVE.sub('', name)
name

'Dr. Vogel '

In [21]:
PERSON_PARTY.match(test_string.rstrip(':')).group(2)

'SPD'

#### Testing find_person_in_db modified

In [23]:
# utterances
# this works! now to implement into parser 
person = test_string
info_dict = {'wp': wp, 'session': session, 'source_type': 'PDF/SP'}
speaker_party = normalize(PARTY_MEMBER_PDF.match(person).group(3))
info_dict['party'] = speaker_party
per = find_person_in_db(person, add_info=info_dict, verbosity=verbosity) 
per

<Person: Dr. Hans-Jochen Vogel>

In [24]:
info_dict['party']

'spd'

In [25]:
# interjections
info_dict = {'wp': wp, 'session': session, 'source_type': 'PDF/POI'}
per = find_person_in_db(person, add_info=info_dict, verbosity=verbosity)

! Warning: Could not distinguish between persons!
For name string: Dr. Vogel 
first name: , surname: Vogel
title: Dr., party: None, position: None, ortszusatz: None
Query: <QuerySet [<Person: Dr. Hans-Jochen Vogel>, <Person: Dr. Vogel>, <Person: Dr. Hans Vogel>, <Person: Dr. HansJochen Vogel>, <Person: Dr. Hans-Joachim Vogel>]>
Clean names: ['Dr. Hans-Jochen Vogel', 'Dr. Vogel', 'Dr. Hans Vogel', 'Dr. HansJochen Vogel', 'Dr. Hans-Joachim Vogel']
Taking first entry of ambiguous results


In [59]:
bracket_sp = 'zum Bundesminister Finanzen (Dr. Vogel [SPD]:'
bracket_sp1 = 'Beifall bei der CDU/CSU und der FDP — Dr. Mechtersheimer [GRÜNE]:'
bracket_sp.split('(')[1]

'Dr. Vogel [SPD]:'

In [70]:
match = re.search('[\(]', bracket_sp)
if match:
    name_br = bracket_sp.split('(')[1]
name_br

'Dr. Vogel [SPD]:'

In [65]:
match1.string

'Beifall bei der CDU/CSU und der FDP — Dr. Mechtersheimer [GRÜNE]:'

In [48]:
minmatch = PARTY_MEMBER_PDF.match(bracket_sp)
minmatch.group(3)

'(Dr. Vogel [SPD]'

In [85]:
POI_SPEAKER = re.compile('[\(—]')
abz = POI_SPEAKER.search(bracket_sp1)

In [86]:
abz

<re.Match object; span=(36, 37), match='—'>